In [ ]:
# IMPORTS

import pandas as pd          # For data manipulation and analysis
import numpy as np           # For numerical operations
import matplotlib.pyplot as plt  # For creating visualizations
import seaborn as sns        # For statistical visualizations (enhances matplotlib)
import os                    # For file and directory operations
import warnings              # To suppress warning messages
from datetime import datetime  # For date and time operations

warnings.filterwarnings('ignore')

In [ ]:
# LOAD DATA

os.chdir("e:/job/ecommerce_data_project")
df = pd.read_csv("data/raw/ecommerce_sales_data.csv")

print("✅ Data loaded successfully!")
print(f"📊 Rows: {df.shape[0]:,}")
print(f"📊 Columns: {df.shape[1]}")

✅ Data loaded successfully!
📊 Rows: 3,500
📊 Columns: 7


In [23]:
# EXPLORE DATA

print("🔍 INITIAL EXPLORATION")
print("="*50)

print("📋 Column information:")
for i, col in enumerate(df.columns, 1):
    dtype = df[col].dtype
    null_count = df[col].isnull().sum()
    unique_count = df[col].nunique()
    print(f"{i:2}. {col:25} | {str(dtype):10} | Null: {null_count:4} | Unique: {unique_count}")

print("\n🧮 Summary statistics:")
print(df.describe())

print("\n🎯 Data types count:")
print(df.dtypes.value_counts())

🔍 INITIAL EXPLORATION
📋 Column information:
 1. Order Date                | object     | Null:    0 | Unique: 1051
 2. Product Name              | object     | Null:    0 | Unique: 10
 3. Category                  | object     | Null:    0 | Unique: 3
 4. Region                    | object     | Null:    0 | Unique: 4
 5. Quantity                  | int64      | Null:    0 | Unique: 9
 6. Sales                     | int64      | Null:    0 | Unique: 2377
 7. Profit                    | float64    | Null:    0 | Unique: 3447

🧮 Summary statistics:
          Quantity         Sales       Profit
count  3500.000000   3500.000000  3500.000000
mean      4.931714   3047.966000   527.047203
std       2.575895   2440.213237   504.139732
min       1.000000     51.000000     6.970000
25%       3.000000   1049.500000   158.695000
50%       5.000000   2350.500000   361.070000
75%       7.000000   4537.000000   729.125000
max       9.000000  10782.000000  2946.930000

🎯 Data types count:
object     4

In [24]:
# MISSING VALUES

print("⚠️ MISSING VALUES ANALYSIS")
print("="*50)

# Calculate missing values
missing = df.isnull().sum()
missing_percent = (missing / len(df)) * 100

# Create DataFrame for better display
missing_df = pd.DataFrame({
    'Missing_Count': missing,
    'Missing_Percent': missing_percent.round(2)
})

# Show only columns with missing values
if missing.sum() > 0:
    print("Columns with missing values:")
    print(missing_df[missing_df['Missing_Count'] > 0])
    
    # Visualize
    plt.figure(figsize=(10, 5))
    bars = plt.barh(missing_df[missing_df['Missing_Count'] > 0].index,
                    missing_df[missing_df['Missing_Count'] > 0]['Missing_Percent'],
                    color='salmon')
    plt.xlabel('Missing Values (%)')
    plt.title('Missing Data by Column')
    plt.xlim(0, 100)
    
    for bar in bars:
        width = bar.get_width()
        plt.text(width + 1, bar.get_y() + bar.get_height()/2,
                f'{width:.1f}%', ha='left', va='center')
    
    plt.tight_layout()
    plt.show()
else:
    print("✅ No missing values found!")

⚠️ MISSING VALUES ANALYSIS
✅ No missing values found!


In [25]:
# HANDLE MISSING VALUES
print("🧹 HANDLING MISSING VALUES")
print("="*50)

# Create a copy for cleaning
df_clean = df.copy()

# Track cleaning actions
cleaning_log = []

# Strategy 1: Numeric columns → fill with median
numeric_cols = df_clean.select_dtypes(include=['int64', 'float64']).columns
for col in numeric_cols:
    if df_clean[col].isnull().any():
        median_val = df_clean[col].median()
        df_clean[col].fillna(median_val, inplace=True)
        null_count = df[col].isnull().sum()
        cleaning_log.append(f"Filled {null_count} missing values in '{col}' with median: {median_val:.2f}")

# Strategy 2: Text columns → fill with mode or 'Unknown'
text_cols = df_clean.select_dtypes(include=['object']).columns
for col in text_cols:
    if df_clean[col].isnull().any():
        null_count = df[col].isnull().sum()
        if null_count < len(df) * 0.3:  # Less than 30% missing
            mode_val = df_clean[col].mode()[0] if not df_clean[col].mode().empty else 'Unknown'
            df_clean[col].fillna(mode_val, inplace=True)
            cleaning_log.append(f"Filled {null_count} missing values in '{col}' with mode: '{mode_val}'")
        else:
            df_clean[col].fillna('Unknown', inplace=True)
            cleaning_log.append(f"Filled {null_count} missing values in '{col}' with 'Unknown'")

print(f"✅ Fixed missing values in {len(cleaning_log)} columns")
print("\nCleaning actions:")
for action in cleaning_log:
    print(f"  • {action}")

print(f"\n📊 Missing values after cleaning: {df_clean.isnull().sum().sum()}")

🧹 HANDLING MISSING VALUES
✅ Fixed missing values in 0 columns

Cleaning actions:

📊 Missing values after cleaning: 0


In [26]:
# REMOVE DUPLICATES

print("🔍 REMOVING DUPLICATES")
print("="*50)

initial_rows = len(df_clean)
df_clean = df_clean.drop_duplicates()
final_rows = len(df_clean)
duplicates_removed = initial_rows - final_rows

if duplicates_removed > 0:
    cleaning_log.append(f"Removed {duplicates_removed} duplicate rows")
    print(f"✅ Removed {duplicates_removed} duplicate rows")
    print(f"Rows before: {initial_rows:,}")
    print(f"Rows after: {final_rows:,}")
else:
    print("✅ No duplicates found")

🔍 REMOVING DUPLICATES
✅ No duplicates found


In [42]:
# CELL: SAVE CLEANED DATA (USING ABSOLUTE PATH)
print("💾 SAVING CLEANED DATA")
print("="*50)

import os

# USE ABSOLUTE PATH - NO GUESSING
project_root = "e:/job/ecommerce_data_project"
print(f"📍 Project root: {project_root}")

# Define paths INSIDE project folder
clean_path = os.path.join(project_root, "data", "processed", "cleaned_ecommerce_data.csv")
sample_path = os.path.join(project_root, "data", "processed", "sample_data.csv")
log_path = os.path.join(project_root, "outputs", "cleaning_log.txt")

print(f"\n📁 Saving to:")
print(f"1. {clean_path}")
print(f"2. {sample_path}")
print(f"3. {log_path}")

# Create directories if they don't exist
os.makedirs(os.path.dirname(clean_path), exist_ok=True)  # Creates data/processed
os.makedirs(os.path.dirname(log_path), exist_ok=True)    # Creates outputs/

# Save the cleaned data
df_clean.to_csv(clean_path, index=False)
print(f"✅ Cleaned data saved: {df_clean.shape[0]:,} rows, {df_clean.shape[1]} columns")

# Save a sample
df_clean.head(100).to_csv(sample_path, index=False)
print(f"✅ Sample data saved: 100 rows")

# Save cleaning log
with open(log_path, 'w') as f:
    f.write("E-COMMERCE DATA CLEANING LOG\n")
    f.write("="*50 + "\n\n")
    f.write(f"Project: E-commerce Sales Data Analysis\n")
    f.write(f"Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Original data: {df.shape[0]:,} rows × {df.shape[1]} columns\n")
    f.write(f"Cleaned data: {df_clean.shape[0]:,} rows × {df_clean.shape[1]} columns\n\n")
    
    f.write("DATA CLEANING ACTIONS:\n")
    f.write("-"*40 + "\n")
    for i, action in enumerate(cleaning_log, 1):
        f.write(f"{i}. {action}\n")
    
    f.write("\nDATA QUALITY SUMMARY:\n")
    f.write("-"*40 + "\n")
    f.write(f"Missing values fixed: {df.isnull().sum().sum() - df_clean.isnull().sum().sum()}\n")
    f.write(f"Duplicates removed: {len(df) - len(df_clean)}\n")
    f.write(f"New features created: {len(df_clean.columns) - len(df.columns)}\n")

print(f"✅ Cleaning log saved")

# Verify files were created
print(f"\n🔍 Verification:")
for path, name in [(clean_path, "Clean data"), (sample_path, "Sample"), (log_path, "Log")]:
    if os.path.exists(path):
        size_kb = os.path.getsize(path) / 1024
        print(f"✅ {name}: {path} ({size_kb:.1f} KB)")
    else:
        print(f"❌ {name} not found: {path}")

print(f"\n📁 Files saved to: {project_root}")

💾 SAVING CLEANED DATA
📍 Project root: e:/job/ecommerce_data_project

📁 Saving to:
1. e:/job/ecommerce_data_project\data\processed\cleaned_ecommerce_data.csv
2. e:/job/ecommerce_data_project\data\processed\sample_data.csv
3. e:/job/ecommerce_data_project\outputs\cleaning_log.txt
✅ Cleaned data saved: 3,500 rows, 7 columns
✅ Sample data saved: 100 rows
✅ Cleaning log saved

🔍 Verification:
✅ Clean data: e:/job/ecommerce_data_project\data\processed\cleaned_ecommerce_data.csv (174.9 KB)
✅ Sample: e:/job/ecommerce_data_project\data\processed\sample_data.csv (5.0 KB)
✅ Log: e:/job/ecommerce_data_project\outputs\cleaning_log.txt (0.5 KB)

📁 Files saved to: e:/job/ecommerce_data_project


In [43]:
# CELL: CHECK PROJECT STATUS
print("📁 CHECKING PROJECT STATUS")
print("="*60)

import os
import pandas as pd

# Project root
project_root = "e:/job/ecommerce_data_project"
print(f"Project location: {project_root}")

# List all important files
print("\n📊 KEY FILES TO COMMIT:")

# 1. Notebooks
print("\n📓 Notebooks:")
notebooks_path = os.path.join(project_root, "notebooks")
if os.path.exists(notebooks_path):
    for file in os.listdir(notebooks_path):
        if file.endswith('.ipynb'):
            size = os.path.getsize(os.path.join(notebooks_path, file)) / 1024
            print(f"  ✅ {file} ({size:.1f} KB)")

# 2. Data files (processed only - raw should be in .gitignore)
print("\n📊 Data files (processed):")
processed_path = os.path.join(project_root, "data", "processed")
if os.path.exists(processed_path):
    for file in os.listdir(processed_path):
        if file.endswith('.csv'):
            size = os.path.getsize(os.path.join(processed_path, file)) / 1024
            print(f"  ✅ {file} ({size:.1f} KB)")

# 3. Outputs
print("\n📝 Output files:")
outputs_path = os.path.join(project_root, "outputs")
if os.path.exists(outputs_path):
    for file in os.listdir(outputs_path):
        size = os.path.getsize(os.path.join(outputs_path, file)) / 1024
        print(f"  ✅ {file} ({size:.1f} KB)")

# 4. Project files
print("\n📄 Project configuration files:")
root_files = ['.gitignore', 'README.md', 'requirements.txt', 'LICENSE', 'project_info.json']
for file in root_files:
    path = os.path.join(project_root, file)
    if os.path.exists(path):
        size = os.path.getsize(path) / 1024
        print(f"  ✅ {file} ({size:.1f} KB)")
    else:
        print(f"  ⚠️  {file} (missing)")

print("\n" + "="*60)
print("✅ Ready for GitHub push!")

📁 CHECKING PROJECT STATUS
Project location: e:/job/ecommerce_data_project

📊 KEY FILES TO COMMIT:

📓 Notebooks:
  ✅ 01_data_cleaning.ipynb.ipynb (13.3 KB)

📊 Data files (processed):
  ✅ cleaned_ecommerce_data.csv (174.9 KB)
  ✅ sample_data.csv (5.0 KB)

📝 Output files:
  ✅ cleaning_log.txt (0.5 KB)
  ✅ plots (0.0 KB)
  ✅ reports (0.0 KB)

📄 Project configuration files:
  ✅ .gitignore (0.4 KB)
  ✅ README.md (2.1 KB)
  ✅ requirements.txt (31.7 KB)
  ⚠️  LICENSE (missing)
  ⚠️  project_info.json (missing)

✅ Ready for GitHub push!


In [44]:
# CELL: CREATE MISSING FILES
print("📝 CREATING MISSING PROJECT FILES")
print("="*60)

import os
import json

project_root = "e:/job/ecommerce_data_project"

# 1. Create requirements.txt if missing
req_path = os.path.join(project_root, "requirements.txt")
if not os.path.exists(req_path):
    requirements = """pandas>=1.5.0
numpy>=1.21.0
matplotlib>=3.5.0
seaborn>=0.11.0
jupyter>=1.0.0
"""
    with open(req_path, 'w') as f:
        f.write(requirements)
    print("✅ Created: requirements.txt")

# 2. Create .gitignore if missing
gitignore_path = os.path.join(project_root, ".gitignore")
if not os.path.exists(gitignore_path):
    gitignore_content = """# Python
__pycache__/
*.py[cod]
*$py.class
*.so
.Python
env/
venv/
.venv/
ENV/
env.bak/
venv.bak/

# Jupyter Notebook
.ipynb_checkpoints/
*.ipynb_checkpoints/

# IDE
.vscode/
.idea/
*.swp
*.swo

# OS
.DS_Store
Thumbs.db

# Data files (large files - don't upload to GitHub)
data/raw/
*.csv
*.xlsx
*.pkl
*.h5
*.feather
*.parquet

# Outputs (can be regenerated)
outputs/

# Git
.git/
"""
    with open(gitignore_path, 'w') as f:
        f.write(gitignore_content)
    print("✅ Created: .gitignore")

# 3. Create project_info.json if missing
info_path = os.path.join(project_root, "project_info.json")
if not os.path.exists(info_path):
    project_info = {
        "project_name": "E-commerce Data Cleaning Portfolio Project",
        "author": "snow6060",
        "description": "Data cleaning pipeline demonstrating Python/Pandas skills for data engineering roles",
        "technologies": ["Python", "Pandas", "NumPy", "Matplotlib", "Seaborn", "Jupyter"],
        "dataset": "E-commerce sales data (2022-2024)",
        "github": "https://github.com/snow6060/ecommerce-data-project",
        "created": "2024",
        "last_updated": "2024"
    }
    with open(info_path, 'w') as f:
        json.dump(project_info, f, indent=2)
    print("✅ Created: project_info.json")

print("\n✅ All project files ready!")

📝 CREATING MISSING PROJECT FILES
✅ Created: project_info.json

✅ All project files ready!
